# Empathic Zoom

Este script utiliza Selenium y la librería Deep Face para detectar las caras de los participantes en una reunión de Zoom web y mostrar las emociones de la reunión.

Para ello, el script abrirá una sesión de Chrome automatizada con Selenium, accederá a la reunión de Zoom utilizando la URL proporcionada, y luego utilizará Deep Face para analizar las expresiones faciales de los participantes.

*(Podemos llegar a usar OpenCV para analizar, ademas de las emociones, los gestos corporales)*

*Por ejemplo:*
- Detección de Movimiento y Postura: Utilizar algoritmos de visión por computadora para detectar el movimiento de las personas y su postura durante la reunión. Un aumento en el movimiento o cambios en la postura pueden indicar un mayor nivel de interés o participación.
- Reconocimiento de Gestos Faciales: Analizar los gestos faciales de los participantes para detectar sonrisas, fruncir el ceño, miradas de atención, entre otros.
- Participación en la Conversación: Analizar la participación verbal de las personas en la reunión, como la cantidad de tiempo que hablan, la frecuencia con la que hacen preguntas o comentarios, etc.

#### Librerías necesarias

- pip install selenium
- pip install deepface

In [3]:
# Importamos librerias necesarias

from selenium import webdriver
from deepface import DeepFace
import time

In [ ]:
# Defino las funciones necesarias

# Funcion para obtener el estado de animo

def obtener_estado_animo(imagen):
    result = DeepFace.analyze(imagen, actions=['emotion'])
    return result['dominant_emotion']

# Funcion para abrir Zoom y obtener las caras

def obtener_caras(url_zoom):
    driver = webdriver.Chrome()
    driver.get(url_zoom)
    time.sleep(10)  # Esperar a que la reunión se cargue completamente

    # Puedes usar la función obtener_estado_animo en este punto para analizar las caras
    # Por ejemplo, puedes capturar la pantalla y luego analizarla con DeepFace
    # Aquí un ejemplo simple:
    screenshot = driver.save_screenshot("screenshot.png")
    estado_animo = obtener_estado_animo("screenshot.png")

    driver.quit()
    return estado_animo


In [ ]:
url_zoom = "URL_DE_TU_REUNION_DE_ZOOM"
estado_animo = obtener_caras(url_zoom)
print("Estado de ánimo de la reunión:", estado_animo)
